In [ ]:
import os
import requests
import time
from duckduckgo_search import DDGS

# --- Configuration ---
SEARCH_QUERY = "Gothic Revival architecture house"
DOWNLOAD_DIRECTORY = "/Users/tawate/Documents/Architecture_Image_Modeling/architectural_style_images/gothic_revival_images"
MAX_IMAGES = 5  # Increased slightly for testing

def download_images():
    print(f"Searching for '{SEARCH_QUERY}' on DuckDuckGo...")

    # Set up headers to look like a real browser
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
    }

    if not os.path.exists(DOWNLOAD_DIRECTORY):
        os.makedirs(DOWNLOAD_DIRECTORY)
        print(f"Created directory: {DOWNLOAD_DIRECTORY}")

    with DDGS() as ddgs:
        # Fetching results
        try:
            search_results = ddgs.images(
                keywords=SEARCH_QUERY,
                region="us-en",
                safesearch="off",
                max_results=MAX_IMAGES
            )
        except Exception as e:
            print(f"Failed to fetch search results: {e}")
            return

        for i, result in enumerate(search_results):
            image_url = result.get("image")
            if not image_url:
                continue

            try:
                print(f"[{i+1}/{MAX_IMAGES}] Downloading: {image_url[:60]}...")
                
                # Pass headers to the request to avoid 403 errors from image hosts
                response = requests.get(image_url, headers=headers, timeout=15)
                response.raise_for_status()

                # Determine extension
                ext = os.path.splitext(image_url.split('?')[0])[1]
                if ext.lower() not in ['.jpg', '.jpeg', '.png', '.webp']:
                    ext = ".jpg"
                
                filename = f"gothic_revival_{i+1}{ext}"
                filepath = os.path.join(DOWNLOAD_DIRECTORY, filename)

                with open(filepath, 'wb') as f:
                    f.write(response.content)
                
                print(f"   -> Saved successfully.")
                
                # Small sleep to prevent aggressive rate limiting
                time.sleep(1)

            except requests.exceptions.RequestException as e:
                print(f"   -> Skip: {e}")
            except Exception as e:
                print(f"   -> Unexpected error: {e}")

    print("\nDownload process finished.")

if __name__ == "__main__":
    download_images()

Searching for 'Gothic Revival architecture house' on DuckDuckGo...


/var/folders/s8/6lj08kz17ls03rcr6ms85hc00000gn/T/ipykernel_14069/1427021875.py:22: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


RatelimitException: https://duckduckgo.com/i.js?o=json&q=Gothic+Revival+architecture+house&l=us-en&vqd=4-216508373693060604467202148159804881958&p=-1&f=%2C%2C%2C%2C%2C 403 Ratelimit

In [2]:
import os
import re
import requests
from duckduckgo_search import DDGS
import time

# --- Configuration ---

# List of house styles transcribed from the image
HOUSE_STYLES = [
    "Saltbox", "Georgian", "Federal", "Greek Revival", "Gothic Revival",
    "Italianate", "Second Empire", "Queen Anne", "Shingle",
    "Richardsonian Romanesque", "Folk Victorian", "Colonial Revival",
    "Cape Cod", "Neo Classical", "Tudor Revival", "Spanish Colonial Revival",
    "French Revival", "Pueblo Revival", "Craftsman", "Modernistic",
    "International", "Ranch/Prairie"
]

# The base directory where all style folders will be created
BASE_DOWNLOAD_DIRECTORY = "architectural_style_images"

# The maximum number of images to download for each style
MAX_IMAGES_PER_STYLE = 50

def sanitize_foldername(name):
    """
    Cleans a string to be used as a valid folder name.
    Replaces spaces and slashes with underscores and removes invalid characters.
    """
    name = name.replace('/', '_').replace(' ', '_')
    name = re.sub(r'[^\w-]', '', name)
    return name

def download_images_for_style(style, base_dir, max_images):
    """
    Searches for images of a given architectural style and downloads them
    into a dedicated sub-folder.
    """
    print(f"\n{'='*20}\nProcessing style: {style}\n{'='*20}")

    style_folder_name = sanitize_foldername(style)
    download_path = os.path.join(base_dir, style_folder_name)
    os.makedirs(download_path, exist_ok=True)
    print(f"Saving images to: {download_path}")

    search_query = f"{style} house architecture"
    print(f"Searching for: '{search_query}'...")

    # --- FIX IS HERE ---
    # Initialize the counter BEFORE the try block to ensure it always exists.
    download_count = 0

    try:
        with DDGS() as ddgs:
            search_results = ddgs.images(
                keywords=search_query,
                region="us-en",
                safesearch="off",
                max_results=max_images
            )
            
            for i, result in enumerate(search_results):
                # The loop will only run if search_results is not empty
                image_url = result.get("image")
                if not image_url:
                    print(f"Skipping result {i+1}: No image URL found.")
                    continue

                try:
                    print(f"[{i+1}/{max_images}] Downloading: {image_url[:70]}...")
                    response = requests.get(image_url, timeout=15)
                    response.raise_for_status()

                    file_ext = os.path.splitext(image_url)[1]
                    if not file_ext or len(file_ext) > 5:
                        content_type = response.headers.get('content-type')
                        if content_type and 'jpeg' in content_type: file_ext = '.jpg'
                        elif content_type and 'png' in content_type: file_ext = '.png'
                        else: file_ext = '.jpg'

                    filename = f"{style_folder_name}_{download_count + 1}{file_ext}"
                    filepath = os.path.join(download_path, filename)

                    with open(filepath, 'wb') as f:
                        f.write(response.content)
                    
                    print(f"   -> Saved as {filepath}")
                    download_count += 1

                except requests.exceptions.RequestException as e:
                    print(f"   -> Error downloading image {i+1}: {e}")
                except Exception as e:
                    print(f"   -> An unexpected error occurred for image {i+1}: {e}")

    except Exception as e:
        print(f"A critical error occurred while searching for {style}: {e}")
    
    # This print statement is now safe, as download_count is guaranteed to exist.
    print(f"\nFinished processing '{style}'. Downloaded {download_count} images.")


def main():
    """
    Main function to loop through all house styles and initiate downloads.
    """
    os.makedirs(BASE_DOWNLOAD_DIRECTORY, exist_ok=True)
    
    total_styles = len(HOUSE_STYLES)
    for index, style in enumerate(HOUSE_STYLES):
        print(f"\n--- Starting style {index + 1} of {total_styles} ---")
        download_images_for_style(
            style=style,
            base_dir=BASE_DOWNLOAD_DIRECTORY,
            max_images=MAX_IMAGES_PER_STYLE
        )
        time.sleep(2)
        
    print("\n\nAll architectural styles have been processed. Scraping complete.")


if __name__ == "__main__":
    main()


--- Starting style 1 of 22 ---

Processing style: Saltbox
Saving images to: architectural_style_images/Saltbox
Searching for: 'Saltbox house architecture'...


/var/folders/s8/6lj08kz17ls03rcr6ms85hc00000gn/T/ipykernel_14069/2218560444.py:54: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


[1/50] Downloading: https://homeshiftteam.com/images/blog/post/132/saltbox-house.webp...
   -> Saved as architectural_style_images/Saltbox/Saltbox_1.webp
[2/50] Downloading: https://www.bhg.com/thmb/PXaxV1xZuCecC1SiUv-zJ42cuaQ=/1866x0/filters:n...
   -> Saved as architectural_style_images/Saltbox/Saltbox_2.jpg
[3/50] Downloading: https://na.rdcpix.com/663211063/8ecb8b5c10a19ccbe1daba7bc38ec77cw-c305...
   -> Saved as architectural_style_images/Saltbox/Saltbox_3.jpg
[4/50] Downloading: https://www.bhg.com/thmb/C8iGOZGBZU9paMTgOa8ClRXb1No=/4000x0/filters:n...
   -> Saved as architectural_style_images/Saltbox/Saltbox_4.jpg
[5/50] Downloading: https://hips.hearstapps.com/hmg-prod/images/saltbox-469219085-16558447...
   -> Saved as architectural_style_images/Saltbox/Saltbox_5.jpg
[6/50] Downloading: https://www.housedigest.com/img/gallery/what-are-saltbox-houses-and-ho...
   -> Saved as architectural_style_images/Saltbox/Saltbox_6.jpg
[7/50] Downloading: https://architecturedetective.com/sa

/var/folders/s8/6lj08kz17ls03rcr6ms85hc00000gn/T/ipykernel_14069/2218560444.py:54: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


A critical error occurred while searching for Georgian: https://duckduckgo.com/i.js?o=json&q=Georgian+house+architecture&l=us-en&vqd=4-304389962064770194470815779864219775229&p=-1&f=%2C%2C%2C%2C%2C 403 Ratelimit

Finished processing 'Georgian'. Downloaded 0 images.

--- Starting style 3 of 22 ---

Processing style: Federal
Saving images to: architectural_style_images/Federal
Searching for: 'Federal house architecture'...


...

A critical error occurred while searching for Modernistic: https://duckduckgo.com/i.js?o=json&q=Modernistic+house+architecture&l=us-en&vqd=4-112063178688884075362217269845884002194&p=-1&f=%2C%2C%2C%2C%2C 403 Ratelimit

Finished processing 'Modernistic'. Downloaded 0 images.

--- Starting style 21 of 22 ---

Processing style: International
Saving images to: architectural_style_images/International
Searching for: 'International house architecture'...


/var/folders/s8/6lj08kz17ls03rcr6ms85hc00000gn/T/ipykernel_14069/2218560444.py:54: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


A critical error occurred while searching for International: https://duckduckgo.com/i.js?o=json&q=International+house+architecture&l=us-en&vqd=4-152373790848580718876169605032949022617&p=-1&f=%2C%2C%2C%2C%2C 403 Ratelimit

Finished processing 'International'. Downloaded 0 images.

--- Starting style 22 of 22 ---

Processing style: Ranch/Prairie
Saving images to: architectural_style_images/Ranch_Prairie
Searching for: 'Ranch/Prairie house architecture'...


/var/folders/s8/6lj08kz17ls03rcr6ms85hc00000gn/T/ipykernel_14069/2218560444.py:54: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


A critical error occurred while searching for Ranch/Prairie: https://duckduckgo.com/i.js?o=json&q=Ranch%2FPrairie+house+architecture&l=us-en&vqd=4-330849883121940918052340105567799920813&p=-1&f=%2C%2C%2C%2C%2C 403 Ratelimit

Finished processing 'Ranch/Prairie'. Downloaded 0 images.


All architectural styles have been processed. Scraping complete.
